In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None
from IPython.display import display, Markdown, HTML
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
csv_urls = {
    'members' : '/content/gdrive/Shareddrives/Unknown Borrowers Project/data/2022-01 data exports v1.2/members/SCoData_members_v1.2_2022-01.csv',
    'books': '/content/gdrive/Shareddrives/Unknown Borrowers Project/data/2022-01 data exports v1.2/books/SCoData_books_v1.2_2022-01.csv',
    'events': '/content/gdrive/Shareddrives/Unknown Borrowers Project/data/2022-01 data exports v1.2/events/SCoData_events_v1.2_2022-01.csv'
}
# load members, books, events as csv
members_df = pd.read_csv(csv_urls['members'])
books_df = pd.read_csv(csv_urls['books'])
events_df = pd.read_csv(csv_urls['events'])

books_df["item_id"] = books_df.uri.apply(
    lambda x: x.split("/")[-2] if pd.notna(x) else None
)

# working with the first member for now...
# generate short ids equivalent to those in member and book dataframes
members_df["id"] = members_df.uri.apply(
    lambda x: x.split("/")[-2]
)

# split multiple members for shared accounts in events
events_df[
    ["first_member_uri", "second_member_uri"]
] = events_df.member_uris.str.split(";", expand=True)

# working with the first member for now...
# generate short ids equivalent to those in member and book dataframes
events_df["member_id"] = events_df.first_member_uri.apply(
    lambda x: x.split("/")[-2]
)
events_df["item_id"] = events_df.item_uri.apply(
    lambda x: x.split("/")[-2] if pd.notna(x) else None
)

events_df['start_datetime'] = pd.to_datetime(events_df.start_date, format='%Y-%m-%d', errors='coerce')
events_df['end_datetime'] = pd.to_datetime(events_df.end_date, format='%Y-%m-%d', errors='coerce')

# get all member-book interactions from events
# shorten URIs for readability in output
borrow_events = events_df[events_df.item_uri.notna()].copy()

# restrict to borrow events only
borrow_events = borrow_events[borrow_events.event_type == 'Borrow'].copy()
# borrow_df = borrow_events[(borrow_events.start_date.isna() == False) & (borrow_events.end_date.isna() == False)]
borrow_events = borrow_events[borrow_events.start_datetime < '1942-01-01']

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
partial_borrowers = pd.read_csv("https://raw.githubusercontent.com/ZoeLeBlanc/unknown_borrowers/main/dataset_generator/data/partial_borrowers_collapsed.csv")

In [ ]:
partial_members = ['raphael-france', 'hemingway-ernest',
                       'colens-fernand', 'kittredge-eleanor-hayden']
partial_df = partial_borrowers[partial_borrowers.member_id.isin(partial_members)]
partial_df['subscription_starttime'] = pd.to_datetime(partial_df['subscription_start'], errors='coerce')
partial_df['subscription_endtime'] = pd.to_datetime(
    partial_df['subscription_end'], errors='coerce')

In [ ]:
lenskit_results = pd.read_csv('/content/gdrive/Shareddrives/Unknown Borrowers Project/data/collapsed_lenskit_model100_scores.csv')
lenskit_results['result_type'] = 'lenskit'

In [ ]:
bipartite_results = pd.read_csv('/content/gdrive/Shareddrives/Unknown Borrowers Project/data/collapsed_bipartite_predictions.csv')
bipartite_results['result_type'] = 'bipartite'

In [ ]:
hinsage_results = pd.read_csv('/content/gdrive/Shareddrives/Unknown Borrowers Project/data/hinsage_link_predictions_100runs.csv')
hinsage_results['result_type'] = 'hinsage'

In [ ]:
memory_cf_results = pd.read_csv('/content/gdrive/Shareddrives/Unknown Borrowers Project/data/collaborative_filtering_predictions.csv')
memory_cf_results['result_type'] = 'memory_cf'

In [ ]:
memory_cf_results = memory_cf_results.rename(columns={'item_uri':'item_id', })

In [ ]:
memory_cf_results['member_period'] = memory_cf_results.member_id + ': ' + memory_cf_results.period

In [ ]:
cols = ['member_id', 'subscription_start', 'subscription_end', 'item_id',
        'mode_score', 'mode_zscore', 'top_score', 'top_zscore',
        'avg_score', 'member_period', 'result_type', 'std_score', 'median_score']
concat_results = pd.concat([hinsage_results[cols], lenskit_results[cols], bipartite_results[cols], memory_cf_results[cols]])
concat_results['coef_variation'] = abs((concat_results.std_score / concat_results.median_score ) )

In [ ]:
# concat_results[concat_results.result_type != 'bipartite'].sort_values(by=['mode_score', 'mode_zscore'], ascending=[False, True])

In [ ]:
# concat_results[(concat_results.result_type == 'hinsage') & (concat_results.member_id.str.contains('hemingway'))].sort_values(by=['coef_variation', 'median_score'], ascending=[True, False])

In [ ]:
# memory_cf_results = memory_cf_results.sort_values(by=['period', 'median', 'coef_var'], ascending=[True, False, True])

# for period in memory_cf_results.period.unique().tolist():
#   sorted_values = memory_cf_results[memory_cf_results.period == period].sort_values(by=['median', 'coef_var'], ascending=[False, True])
#   print(sorted_values[sorted_values.formatted_chart_title.str.contains("Periodical") == False][0:10][['formatted_chart_title', 'median', 'coef_var']])
#   top_values.extend(sorted_values[0:10].formatted_chart_title.tolist())

In [ ]:
# top_values = []
# for period in memory_cf_results.period.unique().tolist():
#   sorted_values = memory_cf_results[memory_cf_results.period == period].sort_values(by=['coef_var','median'], ascending=[True, False])
#   print(sorted_values[0:10][['formatted_chart_title', 'median', 'coef_var']])
#   top_values.extend(sorted_values[0:10].formatted_chart_title.tolist())

In [ ]:
title_lookup = {row.item_id: row.title for row in books_df.itertuples()}

In [ ]:
popular_current = pd.read_csv('/content/gdrive/Shareddrives/Unknown Borrowers Project/data/popular_current.csv')

In [ ]:
popular_all = pd.read_csv('/content/gdrive/Shareddrives/Unknown Borrowers Project/data/popular_alltime.csv')

In [ ]:
def sample_scores(df, get_top, numb_of_books, metrics):
  periods = df.member_period.unique().tolist()
  visualize_df = []
  for period in periods:
    final_books = []
    rows = df[df.member_period == period]
    results = rows.result_type.unique().tolist()
    for result in results:
      final_rows = rows[rows.result_type == result]
      if get_top:
        final_rows = final_rows.sort_values(by=metrics, ascending=[False, True])
        books=final_rows[0:numb_of_books].item_id.unique().tolist()
      else:
        books = rows.item_id.sample(n=numb_of_books).reset_index()
        books=books.item_id.unique().tolist()

      increment = numb_of_books
      while len(books) < numb_of_books:
        increment = increment + 1
        books=final_rows[0:increment].item_id.unique().tolist()

      final_books.extend(books)

    visualize_df.append(rows[rows.item_id.isin(set(final_books))])
  final_df = pd.concat(visualize_df)
  return final_df

In [ ]:
comparisons = [['median_score', 'std_score'], ['mode_score', 'mode_zscore'], ['top_score', 'top_zscore']]
get_top = True
df = concat_results.copy()
numb_of_books = 10
final_rows = []
charts = []
for metrics in comparisons:
  rows = sample_scores(df, get_top, numb_of_books, metrics)
  rows['comparison_metric'] = metrics[0]
  chart = alt.Chart(rows).mark_circle().encode(
    x=metrics[0],
    y=metrics[1],
    color='member_period:N',
    tooltip=['result_type', 'member_id', 'item_id', 'coef_variation'],
    row='result_type'
  )
  final_rows.append(rows)
  charts.append(chart)

In [ ]:
alt.hconcat(*charts)

alt.HConcatChart(...)

In [ ]:
def get_formatted_titles(sorted_values, numb_of_preds):
  titles = []
  for i in sorted_values[0:numb_of_preds].item_id.tolist():
    item = books_df[books_df.item_id == i]
    if item.author.isna().any() == False:
      author = ' '.join(item.author.str.split(',').values[0][::-1])
      # author = ', ' + author
    else: 
      author = '(Periodical)'
    title = item.title.values[0]
    titles.append(f"*{title}*,<br>{author}")
  return titles

In [ ]:
result_types = ['lenskit', 'memory_cf']
subset_results = concat_results[(concat_results.member_id == 'hemingway-ernest') & (concat_results.result_type.isin(result_types))]

In [ ]:
n = len(subset_results)
formatted_titles = get_formatted_titles(subset_results, n)

In [ ]:
subset_results['formatted_title'] = formatted_titles

In [ ]:
subset_results[subset_results.formatted_title.str.contains('Instigations of Ezra Pound')][0:1].formatted_title.values[0]

'*Instigations of Ezra Pound: Together with an Essay on the Chinese Written Character, by Ernest Fenollosa*,<br> Ernest  Ezra;Fenollosa Pound'

In [ ]:
periods = subset_results.member_period.unique().tolist()
numb = 10
periodical_flag = False
for period in periods:
  cf_rows = subset_results[(subset_results.member_period == period) & (subset_results.result_type == 'memory_cf') &(subset_results.formatted_title.str.contains('Periodical') == periodical_flag) ].sort_values(by=['median_score', 'coef_variation'], ascending=[False, True])
  lk_rows = subset_results[(subset_results.member_period == period) & (subset_results.result_type == 'lenskit')& (subset_results.formatted_title.str.contains('Periodical') == periodical_flag)].sort_values(by=['coef_variation', 'median_score'], ascending=[True, False])
  merged_cf = pd.merge(cf_rows[0:numb][['median_score', 'item_id', 'coef_variation', 'formatted_title', 'member_period']], books_df[['item_id', 'borrow_count', 'event_count' ]], on='item_id', how='left')
  print('cf', merged_cf[['formatted_title', 'borrow_count', 'event_count']])
  merged_lk = pd.merge(lk_rows[0:numb][['median_score', 'item_id', 'coef_variation', 'formatted_title', 'member_period']], books_df[['item_id', 'borrow_count', 'event_count' ]], on='item_id', how='left')
  print('lk', merged_lk[['formatted_title', 'borrow_count', 'event_count']])

cf                                      formatted_title  borrow_count  event_count
0  *Thomas Hardy: A Critical Study*,<br> Lascelle...             2            2
1                    *Since Cézanne*,<br> Clive Bell             4            4
2                     *The Clash*,<br> Storm Jameson             3            3
3           *Samuel Butler*,<br> Henry Festing Jones             3            4
4  *The Daniel Jazz and Other Poems*,<br> Vachel ...             2            2
5  *George Gissing: A Critical Study*,<br> Frank ...             5            5
6  *Washington and the Hope of Peace*,<br> H. G. ...             2            2
7      *The Life of Walter Pater*,<br> Thomas Wright             4            4
8             *Shakespeare*,<br> William Shakespeare             7            7
9  *Instigations of Ezra Pound: Together with an ...             8            8
lk                                      formatted_title  borrow_count  event_count
0           *Beyond the Horizon*,<

In [ ]:

book_ids = books_df.item_id.unique().tolist()
subset_popular_all = popular_all[(popular_all.member_id == 'hemingway-ernest')].reset_index()
subset_popular_all = subset_popular_all[subset_popular_all.item_id.isin(book_ids)]
n = len(subset_popular_all)


formatted_titles = get_formatted_titles(subset_popular_all, n)
subset_popular_all['formatted_title'] = formatted_titles

subset_popular_current = popular_current[(popular_current.member_id == 'hemingway-ernest')]
subset_popular_current = subset_popular_current[subset_popular_current.item_id.isin(book_ids)]
n = len(subset_popular_current)
formatted_titles = get_formatted_titles(subset_popular_current, n)
subset_popular_current['formatted_title'] = formatted_titles



In [ ]:
# sort_metrics =['avg_score', 'std_score']
members = subset_results.member_id.unique().tolist()
result_types = subset_results.result_type.unique().tolist()
result_types = ['lenskit', 'memory_cf']
last_member = None
numb_of_preds = 10
popular_preds = 20
top_rows = []
for index, row in partial_df[partial_df.member_id == 'hemingway-ernest'].iterrows():
  if row.member_id != last_member:
      display(HTML("<h3>%s</h3>" % row.member_id))
      display(HTML("<p>%d known borrows</p>" % row.known_borrows))
  display(HTML("<p>subscription: %s – %s</p>" % (row.subscription_starttime.strftime("%Y-%m-%d"), row.subscription_endtime.strftime("%Y-%m-%d"))))
  result_data = {}
  popular_data = {}
  popular_all_rows = subset_popular_all[(subset_popular_all.member_id == row.member_id)& (subset_popular_all.subscription_start == row.subscription_start) & (subset_popular_all.subscription_end == row.subscription_end) & (subset_popular_all.formatted_title.str.contains('Periodical') == False)].sort_values(by='score', ascending=False)
  popular_data['popular (all time)'] = get_formatted_titles(popular_all_rows, numb_of_preds)
  popular_data['popular scores (all time)'] = popular_all_rows[0:numb_of_preds].score.tolist()
  popular_current_rows = subset_popular_current[(subset_popular_current.member_id == row.member_id)& (subset_popular_current.subscription_start == row.subscription_start) & (subset_popular_current.subscription_end == row.subscription_end) & (subset_popular_current.formatted_title.str.contains('Periodical') == False)].sort_values(by='score', ascending=False)
  popular_data['popular (current)'] = get_formatted_titles(popular_current_rows, numb_of_preds)
  popular_data['popular scores (current)'] = popular_current_rows[0:numb_of_preds].score.tolist()
  # print(popular_data)
  for result in result_types:
    selected_rows = subset_results[(subset_results.member_id == row.member_id) & (subset_results.subscription_start == row.subscription_start) & (subset_results.subscription_end == row.subscription_end) & (subset_results.result_type == result)]
    if ((result == 'bipartite' ) or (result == 'memory_cf')):
      sorted_values = selected_rows.sort_values(by=['median_score', 'coef_variation'], ascending=[False, True])
    else:
      sorted_values = selected_rows.sort_values(by=['coef_variation', 'median_score'], ascending=[True, False])
    selected_values = sorted_values[sorted_values.formatted_title.str.contains('Periodical') == False]
    # selected_values = sorted_values
    result_data[result + '_predicted_item'] = [i for i in selected_values[0:numb_of_preds].formatted_title.tolist()]
    # result_data[result + '_predicted_item'] = get_formatted_titles(sorted_values, numb_of_preds)
    result_data[result + '_coef_variation'] = [i for i in selected_values[0:numb_of_preds]['coef_variation'].tolist()]
    result_data[result + '_median_score'] = [i for i in selected_values[0:numb_of_preds]['median_score'].tolist()]
    # sorted_values = selected_rows.sort_values(by=['avg_score'], ascending=[False])
    # result_data[result + '_avg_predicted_item'] = [title_lookup[i] for i in sorted_values[0:numb_of_preds].item_id.tolist()]
    # result_data[result + '_avg_score'] = sorted_values[0:numb_of_preds]['avg_score'].tolist()
    # result_data[result + '_avg_coef'] = sorted_values[0:numb_of_preds]['coef_variation'].tolist()
  table_data = {**result_data, **popular_data}
  # print(table_data)
  table = pd.DataFrame(data=table_data )
  table['member_period'] = row.member_id + ': ' + row.subscription_start + '/' + row.subscription_end
  table['member_id'] = row.member_id
  table['period'] = row.subscription_start + '/' + row.subscription_end
  top_rows.append(table)
  display(HTML(table.to_html(index=False)))
  last_member = row.member_id
    

lenskit_predicted_item,lenskit_coef_variation,lenskit_median_score,memory_cf_predicted_item,memory_cf_coef_variation,memory_cf_median_score,popular (all time),popular scores (all time),popular (current),popular scores (current),member_period,member_id,period
"*Beyond the Horizon*,<br> Eugene O'Neill",0.256911,0.575106,"*Thomas Hardy: A Critical Study*,<br> Lascelles Abercrombie",0.689353,0.285434,"*A Portrait of the Artist as a Young Man*,<br> James Joyce",50.0,"*Dubliners*,<br> James Joyce",13,hemingway-ernest: 1921-12-28/1922-11-08,hemingway-ernest,1921-12-28/1922-11-08
"*Mountain Blood: A Novel*,<br> Joseph Hergesheimer",0.274241,0.197444,"*Since Cézanne*,<br> Clive Bell",0.689353,0.285434,"*Dubliners*,<br> James Joyce",45.0,"*A Portrait of the Artist as a Young Man*,<br> James Joyce",10,hemingway-ernest: 1921-12-28/1922-11-08,hemingway-ernest,1921-12-28/1922-11-08
"*Enjoyment of Poetry*,<br> Max Eastman",0.306714,0.374825,"*The Clash*,<br> Storm Jameson",0.689353,0.285434,"*Pointed Roofs (Pilgrimage 1)*,<br> Dorothy M. Richardson",40.0,"*Erewhon*,<br> Samuel Butler",8,hemingway-ernest: 1921-12-28/1922-11-08,hemingway-ernest,1921-12-28/1922-11-08
"*The Shadow Line: A Confession*,<br> Joseph Conrad",0.333482,0.542490,"*Samuel Butler*,<br> Henry Festing Jones",0.689353,0.285434,"*The Garden Party and Other Stories*,<br> Katherine Mansfield",33.0,"*Exiles*,<br> James Joyce",8,hemingway-ernest: 1921-12-28/1922-11-08,hemingway-ernest,1921-12-28/1922-11-08
"*Nocturne*,<br> Frank Swinnerton",0.363111,-0.503155,"*The Daniel Jazz and Other Poems*,<br> Vachel Lindsay",0.689353,0.285434,"*Bliss and Other Stories*,<br> Katherine Mansfield",31.0,"*Heartbreak House: A Fantasia in the Russian Manner on English Themes*,<br> George Bernard Shaw",7,hemingway-ernest: 1921-12-28/1922-11-08,hemingway-ernest,1921-12-28/1922-11-08
"*Joanna Godden*,<br> Sheila Kaye-Smith",0.372761,0.340152,"*George Gissing: A Critical Study*,<br> Frank Swinnerton",0.689353,0.285434,"*Exiles*,<br> James Joyce",30.0,"*Plays: Pleasant and Unpleasant*,<br> George Bernard Shaw",7,hemingway-ernest: 1921-12-28/1922-11-08,hemingway-ernest,1921-12-28/1922-11-08
"*Tales of Wonder*,<br> Lord Dunsany",0.381612,0.297436,"*Washington and the Hope of Peace*,<br> H. G. Wells",0.689353,0.285434,"*Women in Love*,<br> D. H. Lawrence",29.0,"*Typhoon*,<br> Joseph Conrad",6,hemingway-ernest: 1921-12-28/1922-11-08,hemingway-ernest,1921-12-28/1922-11-08
"*Henry James*,<br> Rebecca West",0.381900,0.304204,"*The Life of Walter Pater*,<br> Thomas Wright",0.689353,0.285434,"*South Wind*,<br> Norman Douglas",27.0,"*The Way of All Flesh*,<br> Samuel Butler",6,hemingway-ernest: 1921-12-28/1922-11-08,hemingway-ernest,1921-12-28/1922-11-08
"*Moon-Calf*,<br> Floyd Dell",0.406374,0.306199,"*Shakespeare*,<br> William Shakespeare",0.710462,0.285305,"*Ulysses*,<br> James Joyce",26.0,"*Cashel Byron's Profession*,<br> George Bernard Shaw",6,hemingway-ernest: 1921-12-28/1922-11-08,hemingway-ernest,1921-12-28/1922-11-08
"*The Happy Prince and Other Tales*,<br> Oscar Wilde",0.415726,0.243187,"*Instigations of Ezra Pound: Together with an Essay on the Chinese Written Character, by Ernest Fenollosa*,<br> Ernest Ezra;Fenollosa Pound",0.815632,0.238664,"*Honeycomb (Pilgrimage 3)*,<br> Dorothy M. Richardson",25.0,"*Under Western Eyes*,<br> Joseph Conrad",6,hemingway-ernest: 1921-12-28/1922-11-08,hemingway-ernest,1921-12-28/1922-11-08


lenskit_predicted_item,lenskit_coef_variation,lenskit_median_score,memory_cf_predicted_item,memory_cf_coef_variation,memory_cf_median_score,popular (all time),popular scores (all time),popular (current),popular scores (current),member_period,member_id,period
"*The Sentry and Other Stories*,<br> Nikola Semyonovich Leskov",0.161543,0.398158,"*Works of Francis Thompson*,<br> Francis Thompson",0.813545,0.569721,"*A Portrait of the Artist as a Young Man*,<br> James Joyce",50.0,"*Dubliners*,<br> James Joyce",24,hemingway-ernest: 1924-03-28/1925-03-28,hemingway-ernest,1924-03-28/1925-03-28
"*English Diaries*,<br> Arthur Ponsonby",0.251403,0.429819,"*The Theory of Beauty*,<br> E. F. Carritt",0.689353,0.428151,"*Dubliners*,<br> James Joyce",45.0,"*A Portrait of the Artist as a Young Man*,<br> James Joyce",20,hemingway-ernest: 1924-03-28/1925-03-28,hemingway-ernest,1924-03-28/1925-03-28
"*Beyond the Horizon*,<br> Eugene O'Neill",0.256911,0.575106,"*Avowals*,<br> George Moore",0.546655,0.354217,"*Pointed Roofs (Pilgrimage 1)*,<br> Dorothy M. Richardson",40.0,"*The Garden Party and Other Stories*,<br> Katherine Mansfield",14,hemingway-ernest: 1924-03-28/1925-03-28,hemingway-ernest,1924-03-28/1925-03-28
"*The Collected Essays and Papers of George Saintsbury, 1875 – 1920*,<br> George Saintsbury",0.263457,0.238436,"*One of Ours*,<br> Willa Cather",0.559369,0.285647,"*A Passage to India*,<br> E. M. Forster",34.0,"*Under Western Eyes*,<br> Joseph Conrad",11,hemingway-ernest: 1924-03-28/1925-03-28,hemingway-ernest,1924-03-28/1925-03-28
"*Mountain Blood: A Novel*,<br> Joseph Hergesheimer",0.274241,0.197444,"*Thomas Hardy: A Critical Study*,<br> Lascelles Abercrombie",0.689353,0.285434,"*The Garden Party and Other Stories*,<br> Katherine Mansfield",33.0,"*Main Street: The Story of Carol Kennicott*,<br> Sinclair Lewis",11,hemingway-ernest: 1924-03-28/1925-03-28,hemingway-ernest,1924-03-28/1925-03-28
"*Great Englishmen of the Sixteenth Century*,<br> Sidney Lee",0.286525,0.054398,"*Lady Susan*,<br> Jane Austen",0.689353,0.285434,"*Bliss and Other Stories*,<br> Katherine Mansfield",31.0,"*Lord Jim*,<br> Joseph Conrad",11,hemingway-ernest: 1924-03-28/1925-03-28,hemingway-ernest,1924-03-28/1925-03-28
"*Cold Harbour*,<br> Francis Brett Young",0.286525,0.054398,"*Since Cézanne*,<br> Clive Bell",0.689353,0.285434,"*Exiles*,<br> James Joyce",30.0,"*Babbitt*,<br> Sinclair Lewis",10,hemingway-ernest: 1924-03-28/1925-03-28,hemingway-ernest,1924-03-28/1925-03-28
"*Lady Susan*,<br> Jane Austen",0.286525,0.054398,"*Lord Byron's Correspondence*,<br> George Gordon Byron Byron",0.689353,0.285434,"*Women in Love*,<br> D. H. Lawrence",29.0,"*Erewhon*,<br> Samuel Butler",10,hemingway-ernest: 1924-03-28/1925-03-28,hemingway-ernest,1924-03-28/1925-03-28
"*Ariosto, Shakespeare and Corneille*,<br> Benedetto Croce",0.286525,0.054398,"*All Things Considered*,<br> G. K. Chesterton",0.689353,0.285434,"*South Wind*,<br> Norman Douglas",27.0,"*Night and Day*,<br> Virginia Woolf",10,hemingway-ernest: 1924-03-28/1925-03-28,hemingway-ernest,1924-03-28/1925-03-28
"*Watch and Ward*,<br> Henry James",0.286525,0.054398,"*The Threshold of Quiet*,<br> Daniel Corkery",0.689353,0.285434,"*Ulysses*,<br> James Joyce",26.0,"*The Way of All Flesh*,<br> Samuel Butler",10,hemingway-ernest: 1924-03-28/1925-03-28,hemingway-ernest,1924-03-28/1925-03-28


In [ ]:
# # sort_metrics =['avg_score', 'std_score']
# members = concat_results.member_id.unique().tolist()
# result_types = concat_results.result_type.unique().tolist()
# result_types = ['lenskit', 'memory_cf']
# last_member = None
# numb_of_preds = 10
# top_rows = []
# for index, row in partial_df[partial_df.member_id == 'hemingway-ernest'].iterrows():
#   if row.member_id != last_member:
#       display(HTML("<h3>%s</h3>" % row.member_id))
#       display(HTML("<p>%d known borrows</p>" % row.known_borrows))
#   display(HTML("<p>subscription: %s – %s</p>" % (row.subscription_starttime.strftime("%Y-%m-%d"), row.subscription_endtime.strftime("%Y-%m-%d"))))
#   result_data = {}
#   popular_data = {}
#   popular_all_rows = popular_all[(popular_all.member_id == row.member_id)& (popular_all.subscription_start == row.subscription_start) & (popular_all.subscription_end == row.subscription_end)].sort_values(by='score', ascending=False)
#   popular_data['popular (all time)'] = get_formatted_titles(popular_all_rows, numb_of_preds)
#   popular_data['popular scores (all time)'] = popular_all_rows[0:numb_of_preds].score.tolist()
#   popular_current_rows = popular_current[(popular_current.member_id == row.member_id)& (popular_current.subscription_start == row.subscription_start) & (popular_current.subscription_end == row.subscription_end)].sort_values(by='score', ascending=False)
#   popular_data['popular (current)'] = get_formatted_titles(popular_current_rows, numb_of_preds)
#   popular_data['popular scores (current)'] = popular_current_rows[0:numb_of_preds].score.tolist()
#   # print(popular_data)
#   for result in result_types:
#     selected_rows = concat_results[(concat_results.member_id == row.member_id) & (concat_results.subscription_start == row.subscription_start) & (concat_results.subscription_end == row.subscription_end) & (concat_results.result_type == result)]
#     if ((result == 'bipartite' ) or (result == 'memory_cf')):
#       sorted_values = selected_rows.sort_values(by=['median_score', 'coef_variation'], ascending=[False, True])
#     else:
#       sorted_values = selected_rows.sort_values(by=['coef_variation', 'median_score'], ascending=[True, False])
#     result_data[result + '_predicted_item'] = get_formatted_titles(sorted_values, numb_of_preds)
#     result_data[result + '_coef_variation'] = [i for i in sorted_values[0:numb_of_preds]['coef_variation'].tolist()]
#     result_data[result + '_median_score'] = [i for i in sorted_values[0:numb_of_preds]['median_score'].tolist()]
#     # sorted_values = selected_rows.sort_values(by=['avg_score'], ascending=[False])
#     # result_data[result + '_avg_predicted_item'] = [title_lookup[i] for i in sorted_values[0:numb_of_preds].item_id.tolist()]
#     # result_data[result + '_avg_score'] = sorted_values[0:numb_of_preds]['avg_score'].tolist()
#     # result_data[result + '_avg_coef'] = sorted_values[0:numb_of_preds]['coef_variation'].tolist()
#   table_data = {**result_data, **popular_data}
#   # print(table_data)
#   table = pd.DataFrame(data=table_data )
#   table['member_period'] = row.member_id + ': ' + row.subscription_start + '/' + row.subscription_end
#   table['member_id'] = row.member_id
#   table['period'] = row.subscription_start + '/' + row.subscription_end
#   top_rows.append(table)
#   display(HTML(table.to_html(index=False)))
#   last_member = row.member_id
    

In [ ]:
final_df = pd.concat(top_rows)
final_df = final_df[['member_period','period','lenskit_predicted_item', 'lenskit_median_score','lenskit_coef_variation', 'memory_cf_predicted_item', 
                     'memory_cf_median_score','memory_cf_coef_variation','popular (current)',
       'popular scores (current)', 'popular (all time)',
       'popular scores (all time)', 'member_id']]
final_df.loc[final_df.memory_cf_predicted_item.str.contains('Ezra'), 'memory_cf_predicted_item'] = "*Instigations of Ezra Pound: Together with an Essay on the Chinese Written Character*,<br>Ezra Pound and Ernest Fenollosa"
final_df.to_csv('/content/gdrive/Shareddrives/Unknown Borrowers Project/data/final_top_results.csv', index=False)

In [ ]:
# periods = final_df.member_period.unique().tolist()
# item_cols = ['hinsage_predicted_item', 'lenskit_predicted_item', 'bipartite_predicted_item',  'popular (all time)', 'popular (current)']

# final_df[item_cols].to_dict(orient='list')
# common_df = []
# for period in periods:
#   rows = final_df[final_df.member_period == period]
#   preds = list(rows[item_cols].to_dict(orient='list').values())
#   result = set(preds[0])
#   commons = {}
#   commons['member_period'] = period
#   common_elements = []
#   for s in preds[1:]:
#     common_elements.extend(list(result.intersection(set(s))))
#   titles = []
#   for i in common_elements:
#     split_title = i.split(', ')[0]
#     split_title = split_title.replace('*', '')
#     item = books_df[books_df.title == split_title]
#     if item.author.isna().any() == False:
#       author = ' '.join(item.author.str.split(', ').values[0][::-1])
#       author = ', ' + author
#     else: 
#       author = ', (Periodical)'
#     title = item.title.values[0]
#     titles.append(f"*{title}*{author}")
#   commons['shared_items'] = titles
#   print(titles)

#   common_df.append(pd.DataFrame([commons]))
    


In [ ]:
# common_df = pd.concat(common_df)
# common_df.explode('shared_items').to_csv('/content/gdrive/Shareddrives/Unknown Borrowers Project/data/shared_items.csv', index=False)

,member_period,shared_items
0,colens-fernand: 1920-04-01/1920-07-07,Exiles
0,colens-fernand: 1920-04-01/1920-07-07,A Portrait of the Artist as a Young Man
0,colens-fernand: 1920-04-01/1920-07-07,Dubliners
0,colens-fernand: 1920-10-30/1927-11-26,A Passage to India
0,colens-fernand: 1920-10-30/1927-11-26,Pointed Roofs (Pilgrimage 1)
0,raphael-france: 1920-04-30/1921-11-17,Pointed Roofs (Pilgrimage 1)
0,raphael-france: 1920-04-30/1921-11-17,Exiles
0,raphael-france: 1920-04-30/1921-11-17,A Portrait of the Artist as a Young Man
0,raphael-france: 1920-04-30/1921-11-17,Women in Love
0,raphael-france: 1920-04-30/1921-11-17,Pointed Roofs (Pilgrimage 1)


In [ ]:
top_results_lenskit = pd.read_csv('/content/gdrive/Shareddrives/Unknown Borrowers Project/data/top_scores_lenskit_model100.csv')

In [ ]:
top_scores_memorycf = pd.read_csv('/content/gdrive/Shareddrives/Unknown Borrowers Project/data/memorycf_top_results.csv')

In [ ]:
subset_lenskit = top_results_lenskit[['member_id', 'subscription_start', 'subscription_end', 'item_id','score','formatted_title', 'period', 'model_run']]

In [ ]:
subset_lenskit = subset_lenskit[subset_lenskit.member_id == 'hemingway-ernest']

In [ ]:
subset_mcf = top_scores_memorycf[['item_uri', 'formatted_chart_title',
       'member_id', 'period', 'subscription_start', 'subscription_end',
       'metric', 'score']]

In [ ]:
subset_lenskit.groupby('period')['item_id'].nunique()

period
 1921-12-28/1922-11-08    33
 1924-03-28/1925-03-28    30
Name: item_id, dtype: int64

In [ ]:
subset_mcf.groupby('period')['item_uri'].nunique()

period
1921-12-28/1922-11-08    30
1924-03-28/1925-03-28    47
Name: item_uri, dtype: int64

In [ ]:
subset_lenskit['period'] = subset_lenskit.period.str.split(' ').str[1]

In [ ]:
from textwrap import wrap

In [ ]:
subset_mcf.loc[subset_mcf.formatted_chart_title.str.contains('Instigations of Ezra Pound'), 'formatted_chart_title'] = 'Instigations of Ezra Pound: Together with an Essay on the Chinese Written Character by Ezra Pound and Ernest Fenollosa'

In [ ]:
subset_mcf['formatted_wrapped_title'] = subset_mcf.formatted_chart_title.apply(wrap, args=[50])
subset_lenskit['formatted_wrapped_title'] = subset_lenskit.formatted_title.apply(wrap, args=[50])

In [ ]:
subset_mcf.loc[subset_mcf.metric == 'cosine', 'metric'] = 'Cosine Distance'
subset_mcf.loc[subset_mcf.metric == 'pearson', 'metric'] = 'Pearson Correlation Coefficient'
subset_mcf.loc[subset_mcf.metric == 'euclidean', 'metric'] = 'Euclidean Distance'

In [ ]:
chart = alt.Chart(subset_mcf).mark_point(opacity=0.8, filled=True, thickness=3).encode(
    y=alt.Y('formatted_chart_title', sort='-x', axis=alt.Axis(title='', labelFontSize=9)),
    x='score',
    color=alt.Color('period:N', legend=alt.Legend(title=["Subscription Period"])),
    # column='member_id:N',
    shape=alt.Shape('metric:N', legend=alt.Legend(title='Method for Collaborative Filtering')),
).properties(
    width=300,
    height=1800,
    title='Top Predictions by Collaborative Filtering Memory-Based Methods'
)
# chart = chart.configure_axisY(
#         titleAngle=0,
#         titleAlign="left",
#         titleY=-10,
#         titleX=-100,
#         labelLimit=1000
#     )

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [ ]:
full_name = members_df[members_df.id == 'hemingway-ernest'].name.values[0]
tickplot = alt.Chart(subset_lenskit).mark_tick(opacity=0.7).encode(
  y=alt.Y('formatted_title', sort='-x', axis=alt.Axis(title="", orient='right', labelFontSize=9)),
  x='score',
  color=alt.Color('period:N', legend=alt.Legend(title=["Subscription Period"], orient='left')),
  # facet='member_period:N'
).properties(
    title=f'Top Predictions by Implicit Matrix Factorization Model',
    width=300,
    height=1800
)

In [ ]:
# tickplot = tickplot.configure_axisY(
#         titleAngle=0,
#         titleAlign="left",
#         titleY=-10,
#         titleX=100,
#         labelLimit=1000
#     )

In [ ]:
alt.hconcat(tickplot, chart).configure_axisY(
        titleAngle=0,
        titleAlign="left",
        titleY=-10,
        titleX=100,
        labelLimit=1000
    )

alt.HConcatChart(...)